In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

device = 'cuda'
# Define the transformation to resize images to 32x32 and convert them to tensors
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# Download and install the Fashion MNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Create data loaders for training and testing
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)

# Define the LeNet architecture
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Create an instance of the LeNet model
model = LeNet().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0

    # Training
    model.train()  # Set the model in training mode
    for i, (images, labels) in enumerate(train_loader, 0):
        inputs = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print statistics every 100 mini-batches
        if i % 100 == 99:
            print(f'Training: Epoch [{epoch+1}/{num_epochs}], Mini-batch: [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.3f}')
            running_loss = 0.0

    # Validation
    model.eval()  # Set the model in evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            inputs = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy = 100 * correct / total
    print(f'Validation: Epoch [{epoch+1}/{num_epochs}], Accuracy: {validation_accuracy:.2f}%')

print('Training and validation finished.')


Training: Epoch [1/5], Mini-batch: [100/1875], Loss: 2.304
Training: Epoch [1/5], Mini-batch: [200/1875], Loss: 2.303
Training: Epoch [1/5], Mini-batch: [300/1875], Loss: 2.301
Training: Epoch [1/5], Mini-batch: [400/1875], Loss: 2.300
Training: Epoch [1/5], Mini-batch: [500/1875], Loss: 2.298
Training: Epoch [1/5], Mini-batch: [600/1875], Loss: 2.296
Training: Epoch [1/5], Mini-batch: [700/1875], Loss: 2.292
Training: Epoch [1/5], Mini-batch: [800/1875], Loss: 2.282
Training: Epoch [1/5], Mini-batch: [900/1875], Loss: 2.257
Training: Epoch [1/5], Mini-batch: [1000/1875], Loss: 2.174
Training: Epoch [1/5], Mini-batch: [1100/1875], Loss: 1.730
Training: Epoch [1/5], Mini-batch: [1200/1875], Loss: 1.227
Training: Epoch [1/5], Mini-batch: [1300/1875], Loss: 1.120
Training: Epoch [1/5], Mini-batch: [1400/1875], Loss: 1.043
Training: Epoch [1/5], Mini-batch: [1500/1875], Loss: 0.979
Training: Epoch [1/5], Mini-batch: [1600/1875], Loss: 0.936
Training: Epoch [1/5], Mini-batch: [1700/1875], L